In [21]:
import pandas as pd

In [22]:
df1 = pd.read_csv('2-fft-malicious-n-0-3-m-1-data.csv')
df2 = pd.read_csv('2-fft-malicious-n-0-3-m-9-data.csv')
df3 = pd.read_csv('2-fft-malicious-n-0-3-m-15-data.csv')

In [23]:
df1 = df1.loc[df1['router'] == 1]
df2 = df2.loc[df2['router'] == 1]
df3 = df3.loc[df3['router'] == 1]

In [24]:
df1 = df1.drop(columns=['router'])
df2 = df2.drop(columns=['router'])
df3 = df3.drop(columns=['router'])

In [25]:
df1.to_csv('2-fft-normal-n-0-3-m-1-data-r1-mal.csv',index=False)
df2.to_csv('2-fft-normal-n-0-3-m-9-data-r1-mal.csv',index=False)
df3.to_csv('2-fft-normal-n-0-3-m-15-data-r1-mal.csv',index=False)

In [26]:
df1 = pd.read_csv('2-fft-normal-n-0-3-m-1-data-r1-mal.csv')
df2 = pd.read_csv('2-fft-normal-n-0-3-m-9-data-r1-mal.csv')
df3 = pd.read_csv('2-fft-normal-n-0-3-m-15-data-r1-mal.csv')

In [27]:
def timecount(df):
    timearr = []
    interval = 99
    count = 0
    for index, row in df.iterrows():
        if row["timestamp"]<=interval:
            count+=1
        else:
            timearr.append([interval+1,count])
            count=1
            interval+=100
    timearr.append([interval+1,count])
    return timearr

In [28]:
def maxcount(timearr,df):
    countarr = []
    increarr = []
    maxarr = []
    for i in range(len(timearr)):
        for cnt in range(timearr[i][1],0,-1):
            countarr.append(cnt)
            maxarr.append(timearr[i][1])
            increment = timearr[i][1] - cnt + 1
            increarr.append(increment)
    df = df.assign(packet_count_decr=countarr)
    df = df.assign(packet_count_incr=increarr)
    df = df.assign(max_packet_count=maxarr)
    return df

In [29]:
df1 = maxcount(timecount(df1),df1)
df2 = maxcount(timecount(df2),df2)
df3 = maxcount(timecount(df3),df3)

In [30]:
df1

,timestamp,outport,inport,cache_coherence_type,flit_id,flit_type,vnet,vc,src_ni,src_router,dst_ni,dst_router,traversal_id,hop_count,current_hop,hop_percentage,enqueue_time,packet_count_decr,packet_count_incr,max_packet_count
0,7,2,0,1,1,4,3,9,1,1,23,7,1,3,0,0.000000,4,8,1,8
1,11,2,4,1,1,4,3,9,0,0,23,7,0,4,1,0.250000,8,7,2,8
2,84,0,1,4,1,1,5,17,23,7,1,1,1,3,3,1.000000,16,6,3,8
3,84,4,2,1,1,4,3,9,3,3,24,8,3,5,2,0.400000,12,5,4,8
4,85,0,1,4,2,2,5,17,23,7,1,1,1,3,3,1.000000,17,4,5,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127845,3270078,4,2,6,2,2,3,9,18,2,20,4,30069,3,1,0.333333,15,4,2,5
127846,3270085,4,2,6,3,2,3,9,18,2,20,4,30069,3,1,0.333333,22,3,3,5
127847,3270092,4,2,6,4,2,3,9,18,2,20,4,30069,3,1,0.333333,29,2,4,5
127848,3270099,4,2,6,5,3,3,9,18,2,20,4,30069,3,1,0.333333,36,1,5,5


In [31]:
def rename(df):
    df['traversal_id'] = df['traversal_id']+1
    df["packet_count_index"] = df["packet_count_decr"]*df["packet_count_incr"]
    df["packet_max_index"] = df["packet_count_index"]*df["max_packet_count"]
    df["port_index"] = df["outport"]*df["inport"]
    df["cache_coherence_flit_index"] = df["cache_coherence_type"]*df["flit_id"]
    df["flit_index"] = df["cache_coherence_flit_index"]*df["flit_type"]
    df["traversal_index"] = df["flit_index"]*df["traversal_id"]
    df["cache_coherence_vnet_index"] = df["cache_coherence_type"]*df["vnet"]
    df["vnet_vc_index"] = df["vnet"]*df["vc"]
    df["vnet_vc_cc_index"] = df["vnet"]*df["cache_coherence_vnet_index"]
    df.head(50)

In [32]:
rename(df1)
rename(df2)
rename(df3)

In [33]:
df1['target'] = 0
df2['target'] = 0
df3['target'] = 0

In [34]:
df1

,timestamp,outport,inport,cache_coherence_type,flit_id,flit_type,vnet,vc,src_ni,src_router,...,packet_count_index,packet_max_index,port_index,cache_coherence_flit_index,flit_index,traversal_index,cache_coherence_vnet_index,vnet_vc_index,vnet_vc_cc_index,target
0,7,2,0,1,1,4,3,9,1,1,...,8,64,0,1,4,8,3,27,9,0
1,11,2,4,1,1,4,3,9,0,0,...,14,112,8,1,4,4,3,27,9,0
2,84,0,1,4,1,1,5,17,23,7,...,18,144,0,4,4,8,20,85,100,0
3,84,4,2,1,1,4,3,9,3,3,...,20,160,8,1,4,16,3,27,9,0
4,85,0,1,4,2,2,5,17,23,7,...,20,160,0,8,16,32,20,85,100,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127845,3270078,4,2,6,2,2,3,9,18,2,...,8,40,8,12,24,721680,18,27,54,0
127846,3270085,4,2,6,3,2,3,9,18,2,...,9,45,8,18,36,1082520,18,27,54,0
127847,3270092,4,2,6,4,2,3,9,18,2,...,8,40,8,24,48,1443360,18,27,54,0
127848,3270099,4,2,6,5,3,3,9,18,2,...,5,25,8,30,90,2706300,18,27,54,0


In [35]:
df2

,timestamp,outport,inport,cache_coherence_type,flit_id,flit_type,vnet,vc,src_ni,src_router,...,packet_count_index,packet_max_index,port_index,cache_coherence_flit_index,flit_index,traversal_index,cache_coherence_vnet_index,vnet_vc_index,vnet_vc_cc_index,target
0,11,2,4,1,1,4,3,9,0,0,...,2,4,8,1,4,4,3,27,9,0
1,84,4,2,1,1,4,3,9,3,3,...,2,4,8,1,4,16,3,27,9,0
2,238,2,4,1,1,4,3,9,0,0,...,1,1,8,1,4,36,3,27,9,0
3,365,2,4,1,1,4,3,9,0,0,...,1,1,8,1,4,48,3,27,9,0
4,386,1,2,1,1,4,3,9,3,3,...,1,1,2,1,4,52,3,27,9,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51289,2508624,4,2,5,2,2,3,9,3,3,...,10,60,8,10,20,419160,15,27,45,0
51290,2508631,4,2,5,3,2,3,9,3,3,...,12,72,8,15,30,628740,15,27,45,0
51291,2508638,4,2,5,4,2,3,9,3,3,...,12,72,8,20,40,838320,15,27,45,0
51292,2508645,4,2,5,5,3,3,9,3,3,...,10,60,8,25,75,1571850,15,27,45,0


In [36]:
df3

,timestamp,outport,inport,cache_coherence_type,flit_id,flit_type,vnet,vc,src_ni,src_router,...,packet_count_index,packet_max_index,port_index,cache_coherence_flit_index,flit_index,traversal_index,cache_coherence_vnet_index,vnet_vc_index,vnet_vc_cc_index,target
0,11,2,4,1,1,4,3,9,0,0,...,2,4,8,1,4,4,3,27,9,0
1,84,4,2,1,1,4,3,9,3,3,...,2,4,8,1,4,16,3,27,9,0
2,238,2,4,1,1,4,3,9,0,0,...,1,1,8,1,4,36,3,27,9,0
3,365,2,4,1,1,4,3,9,0,0,...,1,1,8,1,4,48,3,27,9,0
4,386,1,2,1,1,4,3,9,3,3,...,1,1,2,1,4,52,3,27,9,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69448,3276626,4,2,6,2,2,3,9,18,2,...,1,1,8,12,24,721584,18,27,54,0
69449,3276633,4,2,6,3,2,3,9,18,2,...,1,1,8,18,36,1082376,18,27,54,0
69450,3276640,4,2,6,4,2,3,9,18,2,...,1,1,8,24,48,1443168,18,27,54,0
69451,3276647,4,2,6,5,3,3,9,18,2,...,1,1,8,30,90,2705940,18,27,54,0


In [37]:
df1.dtypes

timestamp                       int64
outport                         int64
inport                          int64
cache_coherence_type            int64
flit_id                         int64
flit_type                       int64
vnet                            int64
vc                              int64
src_ni                          int64
src_router                      int64
dst_ni                          int64
dst_router                      int64
traversal_id                    int64
hop_count                       int64
current_hop                     int64
hop_percentage                float64
enqueue_time                    int64
packet_count_decr               int64
packet_count_incr               int64
max_packet_count                int64
packet_count_index              int64
packet_max_index                int64
port_index                      int64
cache_coherence_flit_index      int64
flit_index                      int64
traversal_index                 int64
cache_cohere

In [20]:
df1.to_csv('2-fft-normal-n-0-3-m-1-data-r1-mal.csv',index=False)
df2.to_csv('2-fft-normal-n-0-3-m-9-data-r1-mal.csv',index=False)
df3.to_csv('2-fft-normal-n-0-3-m-15-data-r1-mal.csv',index=False)